In [1]:
from agents.test_agent import app, researcher, model
from langchain.globals import set_verbose, set_debug
set_debug(True)


# set_verbose(True)

2025-03-15 17:36:40 | INFO     | mcp_suite.servers.qa:<module>:61 - Logging initialized. Log file: /Users/andrew/saga/mcp-suite/src/mcp_suite/servers/qa/logs/saagalint.log


In [8]:
from mcp_suite.servers.qa.agents.service.pylint import get_pylint_files


path = "src/mcp_suite/servers/qa/service/"
results = get_pylint_files(path)
results

{'src/mcp_suite/servers/qa/service/coverage.py',
 'src/mcp_suite/servers/qa/service/flake8.py',
 'src/mcp_suite/servers/qa/service/pylint.py',
 'src/mcp_suite/servers/qa/service/pytest.py',
 'src/mcp_suite/servers/qa/service/tests/test_coverage_service.py',
 'src/mcp_suite/servers/qa/service/tests/test_pytest_service.py'}

In [9]:
from mcp_suite.servers.qa.agents.service.pylint import get_linting_errors

def generate_prompts():
    errors_list = []
    for file in results:
        # Get linting errors for each file
        errors = get_linting_errors(file)
        errors_list.append(errors)
    
    # Generate prompts for files with errors
    prompts = [
        {"messages":{"role":"user", "content":f"In {file} fix the following error {errors[0]}"}}
        for file, errors in zip(results, errors_list)
        if errors
    ]
    return prompts

# Run the function synchronously
prompts = generate_prompts()
prompts

[{'messages': {'role': 'user',
   'content': 'In src/mcp_suite/servers/qa/service/pytest.py fix the following error R0912-too-many-branches\nLine 17: Too many branches (15/12) (too-many-branches)'}},
 {'messages': {'role': 'user',
   'content': "In src/mcp_suite/servers/qa/service/tests/test_pytest_service.py fix the following error E1135-unsupported-membership-test\nLine 183: Value 'result.error' doesn't support membership test (unsupported-membership-test)\nLine 203: Value 'result.error' doesn't support membership test (unsupported-membership-test)\nLine 221: Value 'result.error' doesn't support membership test (unsupported-membership-test)"}},
 {'messages': {'role': 'user',
   'content': 'In src/mcp_suite/servers/qa/service/tests/test_coverage_service.py fix the following error W0719-broad-exception-raised\nLine 293: Raising too general exception: Exception (broad-exception-raised)'}},
 {'messages': {'role': 'user',
   'content': 'In src/mcp_suite/servers/qa/service/flake8.py fix th

In [10]:
prompts[0]

{'messages': {'role': 'user',
  'content': 'In src/mcp_suite/servers/qa/service/pytest.py fix the following error R0912-too-many-branches\nLine 17: Too many branches (15/12) (too-many-branches)'}}

In [5]:
from mcp_suite.servers.qa.service.pylint_agent.service.pylint_graph import coding_agent,app

In [6]:
results = await app.abatch(prompts, config={"configurable": {"thread_id": "1"},"recursion_limit": 45, "run_name": "Pylint_Agent"})

[chain/start] [chain:Pylint_Agent] Entering Chain run with input:
{
  "messages": {
    "role": "user",
    "content": "In src/mcp_suite/servers/qa/service/pytest.py fix the following error R0912-too-many-branches\nLine 17: Too many branches (15/12) (too-many-branches)"
  }
}[chain/start] [chain:Pylint_Agent] Entering Chain run with input:
{
  "messages": {
    "role": "user",
    "content": "In src/mcp_suite/servers/qa/service/tests/test_pytest_service.py fix the following error C0303-trailing-whitespace\nLine 289: Trailing whitespace (trailing-whitespace)\nLine 290: Trailing whitespace (trailing-whitespace)\nLine 295: Trailing whitespace (trailing-whitespace)"
  }
}

[chain/start] [chain:Pylint_Agent] Entering Chain run with input:
{
  "messages": {
    "role": "user",
    "content": "In src/mcp_suite/servers/qa/service/pylint.py fix the following error R1718-consider-using-set-comprehension\nLine 105: Consider using a set comprehension (consider-using-set-comprehension)"
  }
}
[chai

GraphRecursionError: Recursion limit of 45 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

In [7]:
results

[{'messages': [HumanMessage(content='In src/mcp_suite/servers/qa/service/pytest.py fix the following error W1514-unspecified-encoding\nLine 46: Using open without explicitly specifying an encoding (unspecified-encoding)\nLine 129: Using open without explicitly specifying an encoding (unspecified-encoding)', additional_kwargs={}, response_metadata={}, id='e06ff6f4-a1b7-43b5-a33a-d6c64f50896e'),
   AIMessage(content=[{'text': "I'll help you fix the W1514-unspecified-encoding errors in the pytest.py file. This error occurs when using the `open()` function without specifying an encoding parameter. Let's first read the file to understand its content and context.", 'type': 'text'}, {'id': 'toolu_01RWSFUUfMx8VnctDLUzR4zZ', 'input': {'file_path': 'src/mcp_suite/servers/qa/service/pytest.py'}, 'name': 'read_file', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01SUFPu2htXY22AiksazXYqs', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'tool_use', 'stop_sequence':

In [8]:
from pathlib import Path
from mcp_suite.servers.qa.utils.git_utils import get_git_root


#@tool
def read_file(file_path: str) -> str:
    """
    Read the contents of a file.

    Args:
        file_path: The path to the file to read

    Returns:
        The contents of the file as a string
    """
    try:
        git_root = get_git_root()
        return (git_root / Path(file_path)).resolve().read_text()
    except Exception as e:
        return f"Error reading file: {str(e)}"

read_file("src/mcp_suite/servers/qa/service/coverage.py")

'"""Coverage service functions for the pytest server."""\n\nimport json\nfrom typing import Any, Dict, List, Union\n\nfrom mcp_suite.servers.qa import logger\nfrom mcp_suite.servers.qa.models.coverage_models import (\n    BranchCoverage,\n    CoverageIssue,\n)\n\n# Remove redundant import and setup since it\'s already done in __init__.py\n# from mcp_suite.servers.dev.config.config import setup_logging\n# setup_logging("services")\n\n\ndef process_coverage_json(\n    coverage_file: str = "./reports/coverage.json", specific_file: str = ""\n) -> List[CoverageIssue]:\n    """\n    Process coverage JSON and extract only files with missing lines or branches.\n    For problematic files, also examine functions and classes.\n\n    Args:\n        coverage_file: Path to the coverage JSON file\n        specific_file: Optional file path to filter results for a specific file\n\n    Returns:\n        A list of CoverageIssue objects\n\n    Raises:\n        FileNotFoundError: If the coverage file doesn

In [1]:
from mcp_suite.servers.qa.service.pylint_agent.tools.pylint import get_next_pylint_error


get_next_pylint_error("src/mcp_suite/servers/qa/service/pylint_agent/tools/pylint.py")


2025-03-16 10:34:37 | INFO     | mcp_suite.servers.qa:<module>:61 - Logging initialized. Log file: /Users/andrew/saga/mcp-suite/src/mcp_suite/servers/qa/logs/saagalint.log


/var/folders/33/gmytc3cd4jv67wdgx63jm8dh0000gn/T/ipykernel_15051/3176600270.py:4: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  get_next_pylint_error("src/mcp_suite/servers/qa/service/pylint_agent/tools/pylint.py")


ValidationError: 2 validation errors for get_next_pylint_error_and_handoff_to_coding_agent
state
  Input should be a valid dictionary [type=dict_type, input_value='src/mcp_suite/servers/qa...t_agent/tools/pylint.py', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/dict_type
tool_call_id
  Field required [type=missing, input_value={'state': 'src/mcp_suite/..._agent/tools/pylint.py'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing